In [12]:
import cv2
from PIL import Image, ImageTk
from math import sqrt
import math 
import numpy as np

# Use this resizing to use small images 

In [2]:
im = Image.open("logo.jpg")
print(im.size)
resized = im.resize((1100, 1100), Image.ANTIALIAS)
print(resized.size)
#print(list(resized.getdata()))
resized.show()

(360, 360)
(1100, 1100)


In [5]:
im = cv2.imread("logo.jpg")
print(im.shape)
rs = cv2.resize(im, (24,24))
print(rs.shape)
cv2.imshow('original',im)
cv2.waitKey(0)
cv2.imshow('resized',rs)
cv2.waitKey(0)

(360, 360, 3)
(24, 24, 3)


-1

In [10]:

def getHist(imgPix):
    """
    Return bgr Histograms
    """
    imgPix = np.array(imgPix)
    rbgHist= cv2.calcHist(images=[imgPix], 
                        channels=[0, 1, 2], 
                        mask=None, 
                        histSize=[2, 2, 2], 
                        ranges=[0, 256, 0, 256, 0, 256])

    return rbgHist
rs = cv2.resize(im, (24,24))
h = getHist(rs)
v = h.flatten()
print(h)
print(v)
print(np.reshape(v, (2,2,2)))
np.array(list(map(np.float32, v)))


[[[ 12.   0.]
  [  1.  15.]]

 [[ 39.   0.]
  [  6. 503.]]]
[ 12.   0.   1.  15.  39.   0.   6. 503.]
[[[ 12.   0.]
  [  1.  15.]]

 [[ 39.   0.]
  [  6. 503.]]]


array([ 12.,   0.,   1.,  15.,  39.,   0.,   6., 503.], dtype=float32)

# Color Descriptors

In [153]:

def getAvgs(imgPix):
        """
        la moyenne, moment d’ordre un.
        :return: (avgR, avgG, avgB)
        """
        imgPix = np.array(imgPix)
        height, width, channels = imgPix.shape
        sz = height*width
        t = [0. for _ in range(3)]
        for i in range(len(imgPix)):
            for j in range(len(imgPix[0])):
                t[0] += imgPix[i][j][0]
                t[1] += imgPix[i][j][1]
                t[2] += imgPix[i][j][2]
        t = [y/sz for y in t]
        print('Mean:', imgPix.shape, imgPix.size)
        print(t, "equals")
        
        return list(imgPix.mean(axis=1).mean(axis=0))
    
def getSTDs(imgPix):
        """
        L'écart type, moment d’ordre deux.
        :return:
        """
        imgPix = np.array(imgPix)
        height, width, channels = imgPix.shape
        sz = height*width
        
        mean = getAvgs(imgPix)
        v = [0. for _ in range(3)]
        for i in range(len(imgPix)):
            for j in range(len(imgPix[0])):
                v[0] += (imgPix[i][j][0] - mean[0])**2
                v[1] += (imgPix[i][j][1] - mean[1])**2
                v[2] += (imgPix[i][j][2] - mean[2])**2
        v = [sqrt(y/sz) for y in v]
        print('STD:')
        print(v, "equals")
        return list(imgPix.std(axis=1).std(axis=0))
    
def getMoments(imgPix):
        """
        moment d’ordre trois.
        :return:
        """
        imgPix = np.array(imgPix)
        height, width, channels = imgPix.shape
        sz = height*width
        
        mean = getAvgs(imgPix)
        v = [0. for _ in range(3)]
        for i in range(len(imgPix)):
            for j in range(len(imgPix[0])):
                v[0] += (imgPix[i][j][0] - mean[0])**3
                v[1] += (imgPix[i][j][1] - mean[1])**3
                v[2] += (imgPix[i][j][2] - mean[2])**3
        v = [ abs((y/sz))**(1/3) for y in v]
        features = []
        features.extend(mean)
        features.extend(getSTDs(imgPix))
        features.extend(v)
        return features

In [154]:
print(getAvgs(rs))

Mean: (24, 24, 3) 1728
[235.69618055555554, 238.15104166666666, 229.22048611111111] equals
[235.69618055555551, 238.1510416666666, 229.22048611111111]


In [155]:
print(getSTDs(rs))

Mean: (24, 24, 3) 1728
[235.69618055555554, 238.15104166666666, 229.22048611111111] equals
STD:
[45.77348045746132, 42.14232374114895, 66.36832983674661] equals
[25.119248212652426, 25.890382902339375, 42.49990600721883]


In [156]:
print(getMoments(rs))

Mean: (24, 24, 3) 1728
[235.69618055555554, 238.15104166666666, 229.22048611111111] equals
Mean: (24, 24, 3) 1728
[235.69618055555554, 238.15104166666666, 229.22048611111111] equals
STD:
[45.77348045746132, 42.14232374114895, 66.36832983674661] equals
[235.69618055555551, 238.1510416666666, 229.22048611111111, 25.119248212652426, 25.890382902339375, 42.49990600721883, 61.81698750779069, 56.532930354293505, 90.90555695466534]


# Texture 

In [29]:
def getGabor(image_file):
        kernel        = cv2.getGaborKernel((21, 21), 8.0, np.pi/4, 10.0, 0.5, 0, ktype=cv2.CV_32F)
        kernel       /= math.sqrt((kernel * kernel).sum())
        image         = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
        filtered_img  = cv2.filter2D(image,    cv2.CV_8UC3, kernel)
        heigth, width = kernel.shape 
    
        cv2.imwrite("{}.jpg".format(1), filtered_img)
        # convert matrix to vector 
        descriptor = cv2.resize(filtered_img, (3*width, 3*heigth), interpolation=cv2.INTER_CUBIC)
        return np.hstack(descriptor)

In [31]:
getGabor("logo.jpg")

array([201, 201, 201, ..., 201, 201, 201], dtype=uint8)

In [13]:
help(cv2.getGaborKernel)

Help on built-in function getGaborKernel:

getGaborKernel(...)
    getGaborKernel(ksize, sigma, theta, lambd, gamma[, psi[, ktype]]) -> retval
    .   @brief Returns Gabor filter coefficients.
    .   
    .   For more details about gabor filter equations and parameters, see: [Gabor
    .   Filter](http://en.wikipedia.org/wiki/Gabor_filter).
    .   
    .   @param ksize Size of the filter returned.
    .   @param sigma Standard deviation of the gaussian envelope.
    .   @param theta Orientation of the normal to the parallel stripes of a Gabor function.
    .   @param lambd Wavelength of the sinusoidal factor.
    .   @param gamma Spatial aspect ratio.
    .   @param psi Phase offset.
    .   @param ktype Type of filter coefficients. It can be CV_32F or CV_64F .



In [22]:
17*17*17

4913

In [23]:
26*26

676

In [ ]:
#signature = open(index, 'r')
            '''for line in signature:
                line = line.replace("[", "").replace("]", "").replace("(", "").replace(")", "").split(",")
                filename = line[0]
                avgs = [float(x) for x in line[1:]]
                # TODO: Add to M tree
                self.addObjectsToTree([filename, avgs])
                print(".", end= " ")'''

# Save to desk
            '''
            indexes = open('indexBase.txt', 'w')
            for i in range(len(self.indexBase)):
                indexes.write(str(self.indexBase[i]))
                indexes.write("\n")
            indexes.close()
            '''